In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import biogeme.biogeme as bio
from biogeme.expressions import Beta, bioLinearUtility, DefineVariable, Plus, Times, bioDraws, PanelLikelihoodTrajectory, MonteCarlo, log
import biogeme.models as models
import biogeme.database as db
import biogeme.messaging as msg
import biogeme.optimization as opt
import biogeme.results as res
import math



C:\Users\wkuni\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# df = pd.read_csv('dataset.csv')
df = pd.read_csv('dataset_with_weather.csv')

○CC:Cloud Cover in oktas  
　TX:Maximum Temperature  
○TG:Mean Temperature  
○RR:Precipitation amount in 0.1mm  
○SS:Sunshine in 0.1 Hours  
　FG:Wind speed in 0.1 m/s  
　Q_ : quality for _ (0='valid'; 1='suspect'; 9='missing')

In [3]:
# pd.set_option('display.max_columns', None)
# df_weather[['travel_YYYY-MM-DD', 'CC', 'Q_CC', 'TX',
#        'Q_TX', 'TG', 'Q_TG', 'RR', 'Q_RR', 'SS', 'Q_SS', 'FG', 'Q_FG']]

#Change weird value(RR:-9999) to 0.
df.loc[df["RR"]==-9999, "RR"] = 0
# df.describe()

In [4]:
# df[["CC", "TG", "RR", "SS", "FG"]]
df["CC"] = (df["CC"] - df["CC"].min()) / (df["CC"].max() - df["CC"].min())
df["TG"] = (df["TG"] - df["TG"].min()) / (df["TG"].max() - df["TG"].min())
df["TX"] = (df["TX"] - df["TX"].min()) / (df["TX"].max() - df["TX"].min())
df["RR"] = (df["RR"] - df["RR"].min()) / (df["RR"].max() - df["RR"].min())
df["SS"] = (df["SS"] - df["SS"].min()) / (df["SS"].max() - df["SS"].min())
df["FG"] = (df["FG"] - df["FG"].min()) / (df["FG"].max() - df["FG"].min())

In [5]:
df["travel_mode"] = df['travel_mode'].astype('category')
df["choice"] = df["travel_mode"].cat.codes
#check counts
print(df["travel_mode"].value_counts())
df["choice"].value_counts()

drive    35808
pt       28605
walk     14268
cycle     2405
Name: travel_mode, dtype: int64


1    35808
2    28605
3    14268
0     2405
Name: choice, dtype: int64

In [6]:
df.describe()
# (df["CC"] - df["CC"].mean()) / df["CC"].std()

,trip_id,household_id,person_n,trip_n,bus_scale,survey_year,travel_year,travel_month,travel_date,day_of_week,...,Q_TX,TG,Q_TG,RR,Q_RR,SS,Q_SS,FG,Q_FG,choice
count,81086.000000,81086.000000,81086.000000,81086.000000,81086.000000,81086.000000,81086.00000,81086.000000,81086.000000,81086.000000,...,81086.000000,81086.000000,81086.000000,81086.000000,81086.000000,81086.000000,81086.0,81086.000000,81086.0,81086.000000
mean,40542.500000,8709.504588,0.803628,1.535284,0.645876,1.985719,2013.18124,6.694201,15.357300,3.955677,...,0.064993,0.519825,0.064993,0.051176,0.034519,0.293983,0.0,0.313700,0.0,1.675036
std,23407.656301,5070.378464,1.055050,1.771189,0.472069,0.814416,0.90097,3.332166,8.744826,1.936428,...,0.246515,0.205228,0.246515,0.098787,0.556311,0.269723,0.0,0.143834,0.0,0.794111
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2012.00000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
25%,20271.250000,4297.000000,0.000000,0.000000,0.000000,1.000000,2012.00000,4.000000,8.000000,2.000000,...,0.000000,0.355311,0.000000,0.000000,0.000000,0.048276,0.0,0.212963,0.0,1.000000
50%,40542.500000,8662.500000,0.000000,1.000000,1.000000,2.000000,2013.00000,7.000000,15.000000,4.000000,...,0.000000,0.534799,0.000000,0.005181,0.000000,0.248276,0.0,0.296296,0.0,2.000000
75%,60813.750000,13035.000000,1.000000,2.000000,1.000000,3.000000,2014.00000,10.000000,23.000000,6.000000,...,0.000000,0.684982,0.000000,0.067358,0.000000,0.462069,0.0,0.388889,0.0,2.000000
max,81085.000000,17615.000000,9.000000,18.000000,1.000000,3.000000,2015.00000,12.000000,31.000000,7.000000,...,1.000000,1.000000,1.000000,1.000000,9.000000,1.000000,0.0,1.000000,0.0,3.000000


In [7]:
print(df[['CC', 'TX', 'TG', 'RR', 'SS', 'FG']].corr())
df["travel_mode"].unique()

          CC        TX        TG        RR        SS        FG
CC  1.000000 -0.329169 -0.246866  0.304649 -0.783622  0.099805
TX -0.329169  1.000000  0.910287 -0.137498  0.489348 -0.164133
TG -0.246866  0.910287  1.000000 -0.087162  0.433946 -0.095322
RR  0.304649 -0.137498 -0.087162  1.000000 -0.287401  0.166933
SS -0.783622  0.489348  0.433946 -0.287401  1.000000 -0.114890
FG  0.099805 -0.164133 -0.095322  0.166933 -0.114890  1.000000


['drive', 'pt', 'walk', 'cycle']
Categories (4, object): ['cycle', 'drive', 'pt', 'walk']

In [69]:
df.corr()

,age,distance,dur_walking,dur_cycling,dur_pt_total,dur_pt_int_total,dur_driving,cost_driving_total,CC,TX,TG,RR,SS,FG,choice
age,1.000000,-0.013698,-0.014250,-0.016859,-0.020160,-0.011923,-0.030452,-0.027944,0.002077,0.010346,0.010272,-0.003082,-0.002373,-0.002488,-0.074930
distance,-0.013698,1.000000,0.996327,0.989013,0.868560,0.663193,0.934096,0.462455,-0.000810,-0.002507,-0.000241,0.003969,-0.003697,0.006609,-0.131708
dur_walking,-0.014250,0.996327,1.000000,0.991539,0.881418,0.675360,0.929954,0.451207,-0.000294,-0.004219,-0.002220,0.004525,-0.004822,0.007144,-0.143016
dur_cycling,-0.016859,0.989013,0.991539,1.000000,0.874636,0.671141,0.937795,0.472503,0.000930,-0.004903,-0.002610,0.004730,-0.004939,0.005692,-0.130628
dur_pt_total,-0.020160,0.868560,0.881418,0.874636,1.000000,0.738463,0.817053,0.334523,-0.005281,-0.007968,-0.006202,0.001626,-0.000586,0.010145,-0.249124
dur_pt_int_total,-0.011923,0.663193,0.675360,0.671141,0.738463,1.000000,0.619356,0.242241,-0.005651,-0.011938,-0.011026,0.003478,-0.000179,0.009695,-0.149864
dur_driving,-0.030452,0.934096,0.929954,0.937795,0.817053,0.619356,1.000000,0.569918,0.005171,-0.005871,-0.001396,0.001634,-0.008603,0.003644,-0.085058
cost_driving_total,-0.027944,0.462455,0.451207,0.472503,0.334523,0.242241,0.569918,1.000000,0.014881,-0.024701,-0.022045,-0.006651,-0.018734,-0.005510,0.083232
CC,0.002077,-0.000810,-0.000294,0.000930,-0.005281,-0.005651,0.005171,0.014881,1.000000,-0.329169,-0.246866,0.304649,-0.783622,0.099805,0.003632
TX,0.010346,-0.002507,-0.004219,-0.004903,-0.007968,-0.011938,-0.005871,-0.024701,-0.329169,1.000000,0.910287,-0.137498,0.489348,-0.164133,-0.004833


In [6]:
df = df.drop('purpose', axis=1)
df = df.drop('faretype', axis=1)
df = df.drop('travel_mode', axis=1)
df = df.drop('travel_YYYY-MM-DD', axis=1)
df = df.drop("fueltype", axis=1)
df = df.drop(['female', 'driving_license', 'car_ownership'], axis=1)
df = df.drop(["trip_id", 'household_id', 'person_n', 'trip_n', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time_linear'], axis=1)
df = df.drop(["bus_scale", "dur_pt_access", "dur_pt_rail", "dur_pt_bus", 
              "dur_pt_int_waiting", "dur_pt_int_walking", "pt_n_interchanges",  
              "cost_driving_fuel", "cost_driving_con_charge", "driving_traffic_percent"], axis=1)
df = df.drop(["Q_CC", "Q_TX", "Q_TG", "Q_RR", "Q_SS", "Q_FG"], axis=1)

In [7]:
database = db.Database('choiceset', df)
globals().update(database.variables)

$$
\begin{equation}
\begin{split}
& V_cycle= ASC_{cycling} + 
           \beta_{time}*duration\_cycling + 
           \beta_{CC_cycle}*CC(Cloud Cover) +              
           \beta_{wind_cycle}*FG(wind_speed) +
           \beta_{temp_cycle}*TG(average Temperture) +
           \beta_{prec_cycle}*RR(precipitation) +
           \beta_{SS_cycle}*SS(sunshine)\\
& V_walk = ASC_{walk} +
           \beta_{time}*duration\_walking +
           \beta_{CC_walk}*CC(Cloud Cover) +
           \beta_{wind_walk}*FG(wind_speed) +
           \beta_{temp_walk}*TG(average Temperture) +
           \beta_{prec_walk}*RR(precipitation) +
           \beta_{SS_walk}*SS(sunshine)\\
& V_pt = ASC_{PT} +
        \beta_{time}*duration\_pt\_total + 
        \beta_{cost} * cost\_transit + 
        \beta_{CC_pt}*CC(Cloud Cover) +
        \beta_{wind_pt}*FG(wind_speed) +
        \beta_{temp_pt}*TG(average Temperture) +
        \beta_{prec_pt}*RR(precipitation) +
        \beta_{SS_pt}*SS(sunshine)\\
& V_drive = 
      \beta_{time}*duration\_driving + 
      \beta_{cost} * cost\_driving\_total\\
\end{split}
\end{equation}
$$

# Model without weather


In [17]:
ASC_CYCLING = Beta('ASC_CYCLING', 0, -10, 10, 0)
ASC_PT = Beta('ASC_PT', 0, -10, 10, 0)
ASC_DRIVING = Beta('ASC_DRIVING', 0, -10, 10, 0)
ASC_WALKING = Beta('ASC_WALKING', 0, -10, 10, 0)
B_TIME_WALKING = Beta('B_TIME_WALKING', 0, -30, 10, 0)
B_TIME_CYCLING = Beta('B_TIME_CYCLING', 0, -10, 10, 0)
B_TIME_DRIVING = Beta('B_TIME_DRIVING', 0, -10, 10, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, -10, 10, 0)
B_COST = Beta('B_COST', 0, -10, 10, 0)

V_cycle = (
           ASC_CYCLING + 
           B_TIME_CYCLING * dur_cycling 
)  

V_drive = (
#       ASC_DRIVING +
      B_TIME_DRIVING * dur_driving +
      B_COST * cost_driving_total 
      )
    
V_pt = (
      ASC_PT +
      B_COST * cost_transit +
      B_TIME_PT * dur_pt_total 
        )
    
V_walk = (
    ASC_WALKING +
    B_TIME_WALKING * dur_walking 
)

V = {0: V_cycle,
     1: V_drive,
     2: V_pt,
     3: V_walk}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

logprob = models.loglogit(V, av, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'LondonMNL_Normal'

# Estimate the parameters
results_Normal_MNL = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results_Normal_MNL.getEstimatedParameters()

print("CC:Cloud Cover in oktas, TX:Maximum Temperature, TG:Mean Temperature, RR:Precipitation, SS:Sunshine ")
print(results_Normal_MNL)

CC:Cloud Cover in oktas, TX:Maximum Temperature, TG:Mean Temperature, RR:Precipitation, SS:Sunshine 

Results for model LondonMNL_Normal
Output file (HTML):			LondonMNL_Normal~03.html
Nbr of parameters:		8
Sample size:			81086
Excluded data:			0
Init log likelihood:		-69855.28
Final log likelihood:		-69855.28
Likelihood ratio test (init):		-0
Rho square (init):			0
Rho bar square (init):			-0.000115
Akaike Information Criterion:	139726.6
Bayesian Information Criterion:	139801
Final gradient norm:		0.2178273
ASC_CYCLING    : -2.76[0.0356 -77.3 0][0.0379 -72.8 0]
ASC_PT         : -0.44[0.0172 -25.6 0][0.0173 -25.4 0]
ASC_WALKING    : 1.94[0.0265 73 0][0.0336 57.5 0]
B_COST         : -0.157[0.00368 -42.5 0][0.00374 -41.8 0]
B_TIME_CYCLING : -5.37[0.105 -51 0][0.116 -46.1 0]
B_TIME_DRIVING : -6.13[0.081 -75.7 0][0.0936 -65.5 0]
B_TIME_PT      : -3.29[0.0569 -57.8 0][0.0605 -54.4 0]
B_TIME_WALKING : -8.26[0.0732 -113 0][0.102 -80.8 0]
('ASC_PT', 'ASC_CYCLING'):	8.33e-05	0.136	61.9	0	9.99e-0

# MNL Model with weather

In [18]:
#CC,TG,RR,SS
ASC_CYCLING = Beta('ASC_CYCLING', 0, -10, 10, 0)
ASC_PT = Beta('ASC_PT', 0, -10, 10, 0)
ASC_DRIVING = Beta('ASC_DRIVING', 0, -10, 10, 0)
ASC_WALKING = Beta('ASC_WALKING', 0, -10, 10, 0)
B_TIME_WALKING = Beta('B_TIME_WALKING', 0, None, 10, 0)
B_TIME_CYCLING = Beta('B_TIME_CYCLING', 0, None, 10, 0)
B_TIME_DRIVING = Beta('B_TIME_DRIVING', 0, None, 10, 0)
B_TIME_PT = Beta('B_TIME_PT', 0, None, 10, 0)
B_COST = Beta('B_COST', 0, -10, 10, 0)

B_FG = Beta('B_FG', 0, -10, 10, 0)
B_TG = Beta('B_TG', 0, -10, 10, 0)
B_RR = Beta('B_RR', 0, -10, 10, 0)
B_CC = Beta('B_CC', 0, -10, 10, 0)
B_SS = Beta("B_SS", 0, -10, 10, 0)


FG_cycle = Beta('FG_cycle', 0, -10, 10, 0)
FG_walk = Beta('FG_walk', 0, -10, 10, 0)
FG_pt = Beta('FG_pt', 0, -10, 10, 0)
FG_drive = Beta('FG_drive', 0, -10, 10, 0)

TG_cycle = Beta('TG_cycle', 0, -10, 10, 0)
TG_walk = Beta('TG_walk', 0, -10, 10, 0)
TG_pt = Beta('TG_pt', 0, -10, 10, 0)
TG_drive = Beta('TG_drive', 0, -10, 10, 0)

# TG_cycle_2 = Beta('TG_cycle_2', 0, -10, 10, 0)
# TG_walk_2 = Beta('TG_walk_2', 0, -10, 10, 0)
# TG_pt_2 = Beta('TG_pt_2', 0, -10, 10, 0)

RR_cycle = Beta('RR_cycle', 0, -10, 10, 0)
RR_walk = Beta('RR_walk', 0, -10, 10, 0)
RR_drive = Beta('RR_drive', 0, -10, 10, 0)
RR_pt = Beta('RR_pt', 0, -10, 10, 0)
CC_cycle = Beta('CC_cycle', 0, -10, 10, 0)
CC_walk = Beta('CC_walk', 0, -10, 10, 0)
CC_pt = Beta('CC_pt', 0, -10, 10, 0)
CC_drive = Beta('CC_drive', 0, -10, 10, 0)
SS_cycle = Beta('SS_cycle', 0, -10, 10, 0)
SS_walk = Beta('SS_walk', 0, -10, 10, 0)
SS_pt = Beta('SS_pt', 0, -10, 10, 0)
SS_drive = Beta('SS_drive', 0, -10, 10, 0)


# #Common weather parameters

In [20]:
###MNL model
V_cycle = (
           ASC_CYCLING + 
           B_TIME_CYCLING * dur_cycling + 
           B_FG * FG +
           B_TG * TG +
           B_RR * RR +
           B_CC * CC +
           B_SS * SS
)  

V_drive = (
#       ASC_DRIVING +
      B_TIME_DRIVING * dur_driving +
      B_COST * cost_driving_total 
#       B_FG* FG +
#       B_TG * TG +
#       B_RR * RR +
#       B_CC * CC +
#       B_SS * SS
      )
    
V_pt = (
      ASC_PT +
      B_COST * cost_transit +
      B_TIME_PT * dur_pt_total +
      
      B_FG * FG +
      B_TG * TG +
      B_RR * RR +
      B_CC * CC +
      B_SS * SS
        )
    
V_walk = (
    ASC_WALKING +
    B_TIME_WALKING * dur_walking + 
    
    B_FG* FG +
    B_TG * TG +
    B_RR * RR +
    
    B_CC * CC +
    B_SS * SS
)

V = {0: V_cycle,
     1: V_drive,
     2: V_pt,
     3: V_walk}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

logprob = models.loglogit(V, av, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'LondonMNL_common_MNL'

# Estimate the parameters
results_common_MNL = biogeme.estimate()

In [21]:
#Nest parameters 
NEST_NOMOTOR = Beta('NEST_NOMOTOR',1,0,10,0)
NEST_MOTOR = Beta('NEST_MOTOR',1,0,10,0)
#List of alternatives
NO_MOTOR=NEST_NOMOTOR, [1,2]
MOTOR = NEST_MOTOR, [0,3]

nests = NO_MOTOR, MOTOR

logprob = models.lognested(V, av, nests, choice)

#Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Nested_London_common_NL01'

#Calculate the null log likelihood
biogeme.calculateNullLoglikelihood(av)

results_common_NL01 = biogeme.estimate()

# Get the results in a pandas table
# pandasResults = results_NL01.getEstimatedParameters()
# pandasResults2=pandasResults
# print(pandasResults)
print(results_common_NL01)


Results for model Nested_London_common_NL01
Output file (HTML):			Nested_London_common_NL01~00.html
Nbr of parameters:		15
Sample size:			81086
Excluded data:			0
Null log likelihood:		-112409.1
Init log likelihood:		-69513.18
Final log likelihood:		-69483.24
Likelihood ratio test (null):		85851.65
Rho square (null):			0.382
Rho bar square (null):			0.382
Likelihood ratio test (init):		59.87815
Rho square (init):			0.000431
Rho bar square (init):			0.000215
Akaike Information Criterion:	138996.5
Bayesian Information Criterion:	139136
Final gradient norm:		0.4450112
ASC_CYCLING    : -4.14[0.0968 -42.8 0][0.102 -40.8 0]
ASC_PT         : -0.65[0.0663 -9.81 0][0.0675 -9.63 0]
ASC_WALKING    : 2.36[0.0669 35.4 0][0.0689 34.3 0]
B_CC           : -0.0846[0.0609 -1.39 0.165][0.0613 -1.38 0.168]
B_COST         : -0.225[0.00736 -30.6 0][0.00761 -29.6 0]
B_FG           : -0.166[0.0745 -2.23 0.0255][0.0746 -2.23 0.0257]
B_RR           : -0.27[0.113 -2.39 0.0168][0.113 -2.38 0.0172]
B_SS          

In [22]:
### NL02
#Nest parameters 
NEST_NOCAR = Beta('NEST_NOCAR',1,1,10,0)
NEST_CAR = Beta('NEST_CAR',1,1,10,0)
#List of alternatives
NO_CAR=NEST_NOCAR, [0,2,3]
CAR = NEST_CAR, [1]

nests = NO_CAR, CAR

logprob = models.lognested(V, av, nests, choice)

#Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Nested_London_common_NL02'

#Calculate the null log likelihood
biogeme.calculateNullLoglikelihood(av)

results_common_NL02 = biogeme.estimate()

# Get the results in a pandas table
# pandasResults = results.getEstimatedParameters()
# pandasResults2=pandasResults
# print(pandasResults)
print(results_common_NL02)


Results for model Nested_London_common_NL02
Output file (HTML):			Nested_London_common_NL02~01.html
Nbr of parameters:		15
Sample size:			81086
Excluded data:			0
Null log likelihood:		-112409.1
Init log likelihood:		-69824.01
Final log likelihood:		-69799.87
Likelihood ratio test (null):		85218.39
Rho square (null):			0.379
Rho bar square (null):			0.379
Likelihood ratio test (init):		48.29312
Rho square (init):			0.000346
Rho bar square (init):			0.000131
Akaike Information Criterion:	139629.7
Bayesian Information Criterion:	139769.3
Final gradient norm:		0.3601698
ASC_CYCLING    : -2.37[0.0683 -34.8 0][0.0729 -32.5 0]
ASC_PT         : -0.303[0.0474 -6.38 1.76e-10][0.0483 -6.27 3.68e-10]
ASC_WALKING    : 1.81[0.0529 34.2 0][0.054 33.5 0]
B_CC           : -0.0596[0.045 -1.32 0.185][0.0447 -1.34 0.182]
B_COST         : -0.158[0.00364 -43.5 0][0.00372 -42.6 0]
B_FG           : -0.123[0.0551 -2.24 0.0251][0.0547 -2.26 0.0241]
B_RR           : -0.219[0.0832 -2.63 0.00849][0.0828 -2.65 0.

In [23]:
MNL=dict(sorted(results_common_MNL.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
a=dict(sorted(results_common_NL01.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
b=dict(sorted(results_common_NL02.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
df_MNL = pd.DataFrame.from_dict(MNL, orient='index', columns=['MNL'])
# df_MNL.append(results_MNL.getGeneralStatistics())
df1 = pd.DataFrame.from_dict(a, orient='index', columns=['NL01'])
df2 = pd.DataFrame.from_dict(b, orient='index', columns=['NL02'])
df3 = pd.concat([df_MNL, df1, df2], axis=1)

#Adding statistic information
df_st_mnl = pd.DataFrame.from_dict(results_common_MNL.getGeneralStatistics(), orient='index', columns=['MNL', 'size'])
df_st_nl1 = pd.DataFrame.from_dict(results_common_NL01.getGeneralStatistics(), orient='index', columns=["NL01", "size1"])
df_st_nl2 = pd.DataFrame.from_dict(results_common_NL02.getGeneralStatistics(), orient='index', columns=["NL02", "size2"])
df_st = pd.concat([df_st_mnl, df_st_nl1, df_st_nl2], axis=1)

df_st = pd.concat([df3, df_st], axis=0)
df_st = df_st.drop(["size", "size1", "size2"], axis=1)
df_st


,MNL,NL01,NL02
B_TIME_WALKING,-8.269192,-10.206740,-7.565336
B_TIME_DRIVING,-6.135545,-9.190804,-5.927164
B_TIME_CYCLING,-5.374656,-5.295214,-5.227382
B_TIME_PT,-3.289630,-4.913494,-3.288985
ASC_CYCLING,-2.729076,-4.139944,-2.373340
ASC_WALKING,1.964360,2.364844,1.811191
ASC_PT,-0.413500,-0.650392,-0.302727
B_RR,-0.218911,-0.269773,-0.219055
B_COST,-0.156704,-0.225118,-0.158448
B_FG,-0.125097,-0.166381,-0.123323


In [27]:
df_st.to_excel("para_common01.xlsx", index=True)

## Individual parameters

In [24]:
V_cycle = (
           ASC_CYCLING + 
           B_TIME_CYCLING * dur_cycling + 
           FG_cycle * FG +
#            TG_cycle_2 * TG * TG +
           TG_cycle * TG +
#            TG_cycle_2 * TX * TX +
#            TG_cycle * TX +
           RR_cycle * RR +
           CC_cycle * CC +
           SS_cycle * SS
)  

V_drive = (
#       ASC_DRIVING +
      B_TIME_DRIVING * dur_driving +
      B_COST * cost_driving_total 
#       TG_drive * TG +
#       RR_drive * RR +
#       CC_drive * CC +
#       SS_drive * SS
      )
    
V_pt = (
      ASC_PT +
      B_COST * cost_transit +
      B_TIME_PT * dur_pt_total +
      FG_pt * FG +
      TG_pt * TG +
#       TG_pt_2 * TX * TX +
#       TG_pt * TX +
      RR_pt * RR +
      CC_pt * CC +
      SS_pt * SS
#       B_FG * FG +
#       B_TG * TG +
#       B_RR * RR +
#       B_CC * CC
        )
    
V_walk = (
    ASC_WALKING +
    B_TIME_WALKING * dur_walking + 
    FG_walk * FG +
#     TG_walk_2 * TG * TG +
    TG_walk * TG +
#     TG_walk_2 * TX * TX +
#     TG_walk * TX +
    RR_walk * RR +
    CC_walk * CC +
    SS_walk * SS
#     B_FG* FG +
#     B_TG * TG +
#     B_RR * RR +
#     B_CC * CC
)

V = {0: V_cycle,
     1: V_drive,
     2: V_pt,
     3: V_walk}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

logprob = models.loglogit(V, av, choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'LondonMNL_CC_FG'

# Estimate the parameters
results_MNL = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results_MNL.getEstimatedParameters()

print("CC:Cloud Cover in oktas, TX:Maximum Temperature, TG:Mean Temperature, RR:Precipitation, SS:Sunshine ")
print(results_MNL)

CC:Cloud Cover in oktas, TX:Maximum Temperature, TG:Mean Temperature, RR:Precipitation, SS:Sunshine 

Results for model LondonMNL_CC_FG
Output file (HTML):			LondonMNL_CC_FG~18.html
Nbr of parameters:		23
Sample size:			81086
Excluded data:			0
Init log likelihood:		-69787.32
Final log likelihood:		-69787.32
Likelihood ratio test (init):		-0
Rho square (init):			0
Rho bar square (init):			-0.00033
Akaike Information Criterion:	139620.6
Bayesian Information Criterion:	139834.6
Final gradient norm:		0.2536151
ASC_CYCLING    : -2.99[0.122 -24.4 0][0.122 -24.4 0]
ASC_PT         : -0.368[0.0509 -7.22 5.2e-13][0.051 -7.2 5.83e-13]
ASC_WALKING    : 1.91[0.0709 27 0][0.072 26.6 0]
B_COST         : -0.157[0.00368 -42.6 0][0.00374 -41.9 0]
B_TIME_CYCLING : -5.37[0.105 -51 0][0.116 -46.1 0]
B_TIME_DRIVING : -6.14[0.0811 -75.7 0][0.0937 -65.5 0]
B_TIME_PT      : -3.29[0.057 -57.8 0][0.0606 -54.3 0]
B_TIME_WALKING : -8.27[0.0733 -113 0][0.102 -80.7 0]
CC_cycle       : -0.358[0.122 -2.93 0.00338][0.

In [ ]:
# dict(sorted(results.getBetaValues().items(), key=lambda x: abs(x[1]), reverse=True))
# type(results.getBetaValues())

# Estimates for nested moedel

##

In [28]:
#Nest parameters 
NEST_NOMOTOR = Beta('NEST_NOMOTOR',1,0,10,0)
NEST_MOTOR = Beta('NEST_MOTOR',1,0,10,0)
#List of alternatives
NO_MOTOR=NEST_NOMOTOR, [1,2]
MOTOR = NEST_MOTOR, [0,3]

nests = NO_MOTOR, MOTOR

logprob = models.lognested(V, av, nests, choice)

#Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Nested_MOTOR'

#Calculate the null log likelihood
biogeme.calculateNullLoglikelihood(av)

results_NL01 = biogeme.estimate()

# Get the results in a pandas table
# pandasResults = results_NL01.getEstimatedParameters()
# pandasResults2=pandasResults
# print(pandasResults)
print(results_NL01)


Results for model Nested_London_Weather01
Output file (HTML):			Nested_London_Weather01~12.html
Nbr of parameters:		25
Sample size:			81086
Excluded data:			0
Null log likelihood:		-112409.1
Init log likelihood:		-69440.43
Final log likelihood:		-69440.43
Likelihood ratio test (null):		85937.27
Rho square (null):			0.382
Rho bar square (null):			0.382
Likelihood ratio test (init):		3.616587e-06
Rho square (init):			2.6e-11
Rho bar square (init):			-0.00036
Akaike Information Criterion:	138930.9
Bayesian Information Criterion:	139163.4
Final gradient norm:		0.3057915
ASC_CYCLING    : -4.34[0.167 -26.1 0][0.167 -26 0]
ASC_PT         : -0.553[0.0775 -7.14 9.27e-13][0.0775 -7.13 9.82e-13]
ASC_WALKING    : 2.32[0.0815 28.5 0][0.0821 28.2 0]
B_COST         : -0.225[0.00739 -30.4 0][0.00766 -29.4 0]
B_TIME_CYCLING : -5.29[0.184 -28.7 0][0.189 -28 0]
B_TIME_DRIVING : -9.17[0.249 -36.8 0][0.246 -37.3 0]
B_TIME_PT      : -4.9[0.14 -35 0][0.138 -35.6 0]
B_TIME_WALKING : -10.2[0.123 -83.1 0][0.13

In [14]:
# sorted(results.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True)

In [29]:
#Nest parameters 
NEST_NOCAR = Beta('NEST_NOCAR',1,1,10,0)
NEST_CAR = Beta('NEST_CAR',1,1,10,0)
#List of alternatives
NO_CAR=NEST_NOCAR, [0,2,3]
CAR = NEST_CAR, [1]

nests = NO_CAR, CAR

logprob = models.lognested(V, av, nests, choice)

#Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Nested_Ind_Car'

#Calculate the null log likelihood
biogeme.calculateNullLoglikelihood(av)

results_NL02 = biogeme.estimate()

# Get the results in a pandas table
# pandasResults = results.getEstimatedParameters()
# pandasResults2=pandasResults
# print(pandasResults)
# print(results_NL02)


Results for model Nested_London_Weather01_Car
Output file (HTML):			Nested_London_Weather01_Car~09.html
Nbr of parameters:		25
Sample size:			81086
Excluded data:			0
Null log likelihood:		-112409.1
Init log likelihood:		-69756.7
Final log likelihood:		-69756.7
Likelihood ratio test (null):		85304.74
Rho square (null):			0.379
Rho bar square (null):			0.379
Likelihood ratio test (init):		-0
Rho square (init):			0
Rho bar square (init):			-0.000358
Akaike Information Criterion:	139563.4
Bayesian Information Criterion:	139796
Final gradient norm:		0.3039357
ASC_CYCLING    : -2.6[0.118 -22 0][0.121 -21.5 0]
ASC_PT         : -0.265[0.0512 -5.18 2.21e-07][0.052 -5.11 3.26e-07]
ASC_WALKING    : 1.77[0.0688 25.8 0][0.068 26.1 0]
B_COST         : -0.159[0.00365 -43.5 0][0.00372 -42.6 0]
B_TIME_CYCLING : -5.23[0.0999 -52.3 0][0.113 -46.2 0]
B_TIME_DRIVING : -5.93[0.0838 -70.7 0][0.0989 -60 0]
B_TIME_PT      : -3.29[0.0552 -59.6 0][0.0587 -56 0]
B_TIME_WALKING : -7.57[0.111 -68.2 0][0.122 -62.1

In [36]:
# df.describe()

In [31]:
MNL=dict(sorted(results_MNL.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
a=dict(sorted(results_NL01.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
b=dict(sorted(results_NL02.getBetaValues().items(), key=lambda x:abs(x[1]), reverse=True))
df_MNL = pd.DataFrame.from_dict(MNL, orient='index', columns=['MNL'])
# df_MNL.append(results_MNL.getGeneralStatistics())
df1 = pd.DataFrame.from_dict(a, orient='index', columns=['NL01'])
df2 = pd.DataFrame.from_dict(b, orient='index', columns=['NL02'])
df3 = pd.concat([df_MNL, df1, df2], axis=1)

#Adding statistic information
df_st_mnl = pd.DataFrame.from_dict(results_MNL.getGeneralStatistics(), orient='index', columns=['MNL', 'size'])
df_st_nl1 = pd.DataFrame.from_dict(results_NL01.getGeneralStatistics(), orient='index', columns=["NL01", "size1"])
df_st_nl2 = pd.DataFrame.from_dict(results_NL02.getGeneralStatistics(), orient='index', columns=["NL02", "size2"])
df_st = pd.concat([df_st_mnl, df_st_nl1, df_st_nl2], axis=1)

df_st = pd.concat([df3, df_st], axis=0)
df_st = df_st.drop(["size", "size1", "size2"], axis=1)
df_st

,MNL,NL01,NL02
B_TIME_WALKING,-8.270228,-1.020008e+01,-7.568910
B_TIME_DRIVING,-6.135298,-9.172250e+00,-5.928235
B_TIME_CYCLING,-5.374597,-5.294389e+00,-5.226454
B_TIME_PT,-3.289741,-4.903609e+00,-3.289643
ASC_CYCLING,-2.985368,-4.344485e+00,-2.603398
ASC_WALKING,1.914228,2.318850e+00,1.774358
TG_cycle,0.902298,1.133102e+00,0.803052
ASC_PT,-0.367780,-5.530695e-01,-0.265423
CC_cycle,-0.357877,-4.982957e-01,-0.315448
RR_pt,-0.248019,-3.754357e-01,-0.247310


In [32]:
df_st.to_excel("df_ind.xlsx", index=True)

# Likelihood ratio test 

In [42]:
from scipy.stats import chi2
LR = 2 * (-69787.315785 + 69855.28)
p_value = 1 - chi2.cdf(LR, 23-8)
p_value

0.0

In [41]:
chi2.cdf(LR, 25-8)
LR

829.7000000000116